In [58]:
import sys,os,json,collections
import tifffile as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np

#ref1 https://github.com/darylclimb/image_affine_transform/blob/master/transformation.ipynb
#https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html
#https://cristianpb.github.io/blog/image-rotation-opencv
#https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/

def rotateAndScale(img, scaleFactor = 0.5, InPlaneRot_Degree = 30):
    (oldY,oldX) = img.shape #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=InPlaneRot_Degree, scale=scaleFactor) #rotate about center of image.
    print(M)
    
    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(InPlaneRot_Degree)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    (tx,ty) = ((newX-oldX)/2,(newY-oldY)/2)
    M[0,2] += tx #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += ty

    rotatedImg = cv2.warpAffine(np.float32(img), M, (int(newX),int(newY)))
    return M, rotatedImg

def rotateScaleTranslate(img, Translation = (1,1), scaleFactor = 0.5, InPlaneRot_Degree = 30):
    (oldY,oldX) = img.shape #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=InPlaneRot_Degree, scale=scaleFactor) #rotate about center of image.
    print(M)
    
    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(InPlaneRot_Degree)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    
    M[0,2] += Translation[0] #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += Translation[1]

    rotatedImg = cv2.warpAffine(np.float32(img), M, (int(newX),int(newY)))
    return M, rotatedImg


def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # centre
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))


def rotate_box(bb, cx, cy, h, w, theta=5):
    new_bb = list(bb)
    for i,coord in enumerate(bb):
        # opencv calculates standard transformation matrix
        theta = np.radians(theta)
        M = cv2.getRotationMatrix2D((cx, cy), theta, 1.0)
        # Grab  the rotation components of the matrix)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])
        # compute the new bounding dimensions of the image
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))
        # adjust the rotation matrix to take into account translation
        M[0, 2] += (nW / 2) - cx
        M[1, 2] += (nH / 2) - cy
        # Prepare the vector to be transformed
        v = [coord[0],coord[1],1]
        # Perform the actual rotation and return the image
        calculated = np.dot(M,v)
        #new_bb[i] = (calculated[0],calculated[1])
    return calculated

In [20]:
img = cv2.imread('DB.tif').sum(-1)
img.shape

(960, 1280)

In [21]:
%matplotlib qt
#Transmatrix, rot_image = rotateAndScale(img, scaleFactor = 2, InPlaneRot_Degree = 5)
rot_image = rotate_bound(np.float32(img), 5)

fig, ax = plt.subplots(2,1)
ax[0].imshow(img)
ax[1].imshow(rot_image)
plt.show()

In [85]:
bb = [[1172,304]]
(h, w) = img.shape[:2]
(cx, cy) = (w // 2, h // 2)

(new_h, new_w) = rot_image.shape[:2]
(new_cx, new_cy) = (new_w // 2, new_h // 2)

reached = rotate_box(bb, cx, cy, h, w, theta=30)
reached

array([1166.72551481,  317.07952893])

In [86]:
diffx = 1231 - reached[0]
diffy = 391 - reached[1]

diffx, diffy

(64.27448518667848, 73.92047106606105)

In [73]:
#desired = 1231,391, diffx = 59

In [74]:
(new_cx-cx)*np.cos(np.radians(15))

37.671107225273666

In [75]:
def get_rotation(angle):
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle), -np.sin(angle), 0],
        [np.sin(angle),  np.cos(angle), 0],
        [0, 0, 1]
    ])
def get_translation(tx, ty):
    return np.array([
        [1, 0, tx],
        [0, 1, ty],
        [0, 0, 1]
    ])

def get_scale(s):
    return np.array([
        [s, 0, 0],
        [0, s, 0],
        [0, 0, 1]
    ])

R1 = get_rotation(0)
T1 = get_translation(0, 0)
S1 = get_scale(0.25)

In [76]:
M = S1@R1@T1
M2D = M[:2]
xS,yS = img.shape

In [79]:
result = cv2.warpAffine(np.float32(img), M2D, (xS,yS), flags=cv2.INTER_LINEAR)
plt.imshow(result)

In [80]:
print(img[10,10])
M2D@[10,10,0]

186


array([2.5, 2.5])

In [82]:
result[3,3]

171.0

In [88]:
np.arcsin(np.radians(24))

0.43221046627437304

In [94]:
np.cos(np.radians(25))

0.9063077870366499

In [97]:
angle = [10,20,30]
xDiff = [1276-1104,1279-1014, 1279-834]
plt.plot(np.radians(angle),xDiff)

In [98]:
self.xWhere = (i-cx)*np.cos(angle)-((j-cy)*np.sin(angle)) +cx
self.yWhere = (j-cy)*np.cos(angle)+((i-cx)*np.sin(angle)) +cy

NameError: name 'i' is not defined